In [56]:
import torch
from torch import nn
import pytorch_lightning as pl
import math

In [60]:
params = {}

In [62]:
hparams = {'accelerator': 'gpu',
           'devices': [0],
           'dtype': torch.float64,
           'precision': 64,
           'loss_terms': [F.mse_loss],
           'loss_coeffs': [1],
           'optimizer': torch.optim.Adam, 
           'learning_rate': 1e-4,
           'batch_size': 100,
           'max_epochs': 5000,
           'early_stopping_patience': 1000000,
           'DeepONet': False,
           'VarMiON': True,
           'NGO': False,
           'Q': 12,
           'Q_L': 268,
           'h': 64, 
           'Petrov-Galerkin': False,
           'bias_NLBranch': False,
           'bias_LBranch': False,
           'NLB_outputactivation': nn.Tanhshrink(),
           'Cholesky': False,
           'scale_invariance': False,
           'norm_basis': True,
           'bound_mus': False,
           '1/theta': False,
           'symgroupavg': False,
           'NOMAD': False}

params['hparams'] = {}
params['hparams'] = hparams

In [262]:
Q = 12
x = torch.rand(hparams['batch_size'],hparams['h'],hparams['h'])

In [263]:
class ReshapeLayer(nn.Module):
    def __init__(self, output_shape):
        super().__init__()
        self.output_shape = output_shape
    
    def forward(self, x):
        y = x.reshape(self.output_shape)
        return y

In [264]:
class NLBranchNet(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.hparams = params['hparams']
        self.layers = nn.ModuleList()
        
        # Adjusted convolutional layers
        self.layers.append(ReshapeLayer((hparams['batch_size'],1,hparams['h'],hparams['h'])))
        self.layers.append(nn.Conv2d(in_channels=1, out_channels=16, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.BatchNorm2d(num_features=16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.layers.append(nn.Conv2d(in_channels=16, out_channels=32, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.BatchNorm2d(num_features=32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.layers.append(nn.Conv2d(in_channels=32, out_channels=16, kernel_size=4, stride=1, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Conv2d(in_channels=16, out_channels=1, kernel_size=4, stride=1, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(ReshapeLayer((hparams['batch_size'],100)))
        self.layers.append(nn.Linear(100,100))
        self.layers.append(nn.ReLU())
        self.layers.append(ReshapeLayer((hparams['batch_size'],1,10,10)))
        self.layers.append(nn.ConvTranspose2d(1, 16, kernel_size=4, stride=1, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.BatchNorm2d(num_features=16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.layers.append(nn.ConvTranspose2d(16, 32, kernel_size=4, stride=1, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.BatchNorm2d(num_features=32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
        self.layers.append(nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.ConvTranspose2d(16, 1, kernel_size=2, stride=2, bias=self.hparams.get('bias_NLBranch', True)))
        self.layers.append(ReshapeLayer((hparams['batch_size'],64,64)))
        if self.hparams['NLB_outputactivation'] is not None:
            self.layers.append(self.hparams['NLB_outputactivation'])

    def forward(self, x):
        for layer in self.layers:
            print(x.shape)
            x = layer(x)
        y = x
        return y

In [265]:
model = NLBranchNet(params)

In [266]:
sum(p.numel() for p in model.parameters())

31412

In [267]:
model

NLBranchNet(
  (layers): ModuleList(
    (0): ReshapeLayer()
    (1): Conv2d(1, 16, kernel_size=(2, 2), stride=(2, 2), bias=False)
    (2): ReLU()
    (3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Conv2d(16, 32, kernel_size=(2, 2), stride=(2, 2), bias=False)
    (5): ReLU()
    (6): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(32, 16, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (8): ReLU()
    (9): Conv2d(16, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (10): ReshapeLayer()
    (11): Linear(in_features=100, out_features=100, bias=True)
    (12): ReLU()
    (13): ReshapeLayer()
    (14): ConvTranspose2d(1, 16, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (15): ReLU()
    (16): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (17): ConvTranspose2d(16, 32, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (18): ReLU()
    (19): BatchN

In [268]:
model(x).shape

torch.Size([100, 64, 64])
torch.Size([100, 1, 64, 64])
torch.Size([100, 16, 32, 32])
torch.Size([100, 16, 32, 32])
torch.Size([100, 16, 32, 32])
torch.Size([100, 32, 16, 16])
torch.Size([100, 32, 16, 16])
torch.Size([100, 32, 16, 16])
torch.Size([100, 16, 13, 13])
torch.Size([100, 16, 13, 13])
torch.Size([100, 1, 10, 10])
torch.Size([100, 100])
torch.Size([100, 100])
torch.Size([100, 100])
torch.Size([100, 1, 10, 10])
torch.Size([100, 16, 13, 13])
torch.Size([100, 16, 13, 13])
torch.Size([100, 16, 13, 13])
torch.Size([100, 32, 16, 16])
torch.Size([100, 32, 16, 16])
torch.Size([100, 32, 16, 16])
torch.Size([100, 16, 32, 32])
torch.Size([100, 16, 32, 32])
torch.Size([100, 1, 64, 64])
torch.Size([100, 64, 64])


torch.Size([100, 64, 64])

In [170]:
sum(p.numel() for p in model.parameters())

12

In [171]:
class NLBranchNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.ModuleList()
        # self.layers.append(ResizeLayer2D(params, input_dim=self.hparams['input_dim'], output_dim=12))
        # self.layers.append(UnsqueezeLayer(params))
        self.layers.append(nn.ConvTranspose2d(in_channels=1, out_channels=16, kernel_size=4, stride=4))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.BatchNorm2d(num_features=16))
        self.layers.append(nn.ConvTranspose2d(in_channels=16, out_channels=32, kernel_size=4, stride=4))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.BatchNorm2d(num_features=32))
        self.layers.append(nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=4, stride=4))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.ConvTranspose2d(in_channels=16, out_channels=1, kernel_size=4, stride=4))
        # self.layers.append(SqueezeLayer(params))
        # self.layers.append(ResizeLayer2D(params, input_dim=72, output_dim=self.hparams['latent_dim']))

    def forward(self, x):
        x = x.unsqueeze(1)
        for layer in self.layers:
            x = layer(x)
        y = x.squeeze()   
        return y

In [172]:
model2 = NLBranchNet()

In [128]:
sum(p.numel() for p in model2.parameters())

17057

In [53]:
model2.forward

tensor([[[[1.2186, 1.2186, 1.2186,  ..., 0.8033, 0.8033, 0.8033],
          [1.2186, 1.2186, 1.2186,  ..., 0.8033, 0.8033, 0.8033],
          [1.2186, 1.2186, 1.2186,  ..., 0.8033, 0.8033, 0.8033],
          ...,
          [0.4716, 0.4716, 0.4716,  ..., 1.3901, 1.3901, 1.3901],
          [0.4716, 0.4716, 0.4716,  ..., 1.3901, 1.3901, 1.3901],
          [0.4716, 0.4716, 0.4716,  ..., 1.3901, 1.3901, 1.3901]]]])